In [1]:
!pip install --quiet--openai langchain langchain-openai


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --quiet--openai


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, api_key=api_key)

In [3]:
# Prompt
prompt = ChatPromptTemplate.from_template("Tell me a joke about {topic}")
parser = StrOutputParser()

# Chain: Prompt - LLM - OutputParser
chain = prompt | llm | parser

### Invoke Text Output

In [5]:
def invoke_joke(topic: str):
    result = chain.invoke({"topic": topic})
    print(result)

### Streaming Text Output

In [7]:
async def stream_joke(topic: str):
    async for chunk in chain.astream({"topic": topic}):
        print(chunk, end="", flush=True)

### Raw Event Streaming

In [9]:
async def stream_events_joke(topic: str):
    async for event in chain.astream_events({"topic": topic}):
        if event["event"] == "on_chat_model_stream":
            print(event["data"]["chunk"].content, end="", flush=True)

### Testing

In [11]:
import nest_asyncio
import asyncio

nest_asyncio.apply()  # Allows nested event loops

async def main():
    topic_entered = input("What topic would you like to hear a joke on? ")
    stream_option = input("Do you want the output to be streamed? [Y/N] ").upper()

    while stream_option not in ["Y", "N"]:
        stream_option = input("Please enter 'Y' or 'N': ").upper()

    if stream_option == "Y":
        await stream_joke(topic_entered)
    else:
        invoke_joke(topic_entered)

In [23]:
await main()

What topic would you like to hear a joke on?  tech
Do you want the output to be streamed? [Y/N]  n


Why did the computer go to therapy?

Because it had too many bytes from its past!


In [13]:
# await stream_events_joke(topic_entered)